# Create reusable programmatic units that can be composed into dashboards and used in notebooks.

In [1]:
    import traitlets, pandas, toolz.curried as toolz, ipywidgets, json, typing, pathlib, IPython, lsst.daf.persistence, lsst.qa.explorer.functors, datashader, matplotlib, operator, param, holoviews.operation.datashader, bokeh

Subselect some parameters.  We can pick and choose which we prefer.

In [2]:
    metric: ['base_Footprint_nPix', 'Gaussian-PSF_magDiff_mmag', 'CircAper12pix-PSF_magDiff_mmag', 'Kron-PSF_magDiff_mmag', 'CModel-PSF_magDiff_mmag', 'traceSdss_pixel', 'traceSdss_fwhm_pixel', 'psfTraceSdssDiff_percent', 'e1ResidsSdss_milli', 'e2ResidsSdss_milli', 'deconvMoments', 'compareUnforced_Gaussian_magDiff_mmag', 'compareUnforced_CircAper12pix_magDiff_mmag', 'compareUnforced_Kron_magDiff_mmag', 'compareUnforced_CModel_magDiff_mmag'] 
    metric: __annotations__['metric'][:10]
    tract: [9615, 9813]
    tract = __annotations__['tract'][0]
    flag: ['calib_psf_used', 'calib_psf_candidate', 'calib_photometry_reserved', 'merge_measurement_i2', 'merge_measurement_i', 'merge_measurement_r2', 'merge_measurement_r', 'merge_measurement_z', 'merge_measurement_y', 'merge_measurement_g', 'merge_measurement_N921', 'merge_measurement_N816', 'merge_measurement_N1010', 'merge_measurement_N387', 'merge_measurement_N515', 'qaBad_flag']
    flag: __annotations__['flag'][:10]
    filter: ['HSC-G', 'HSC-I', 'HSC-R', 'HSC-Z', 'HSC-Y'] 
    filter = __annotations__['filter'][1]
    table: ['analysisCoaddTable_forced']
    table = __annotations__['table'][0]
    target: ['/project/tmorton/tickets/DM-20015/RC2_w18'] 
    target = __annotations__['target'][0]
    composite: typing.Dict[str, lsst.qa.explorer.functors.Functor] = {'label': lsst.qa.explorer.functors.StarGalaxyLabeller(), 'ra': lsst.qa.explorer.functors.RAColumn(), 'dec': lsst.qa.explorer.functors.DecColumn()}

Connect to parquest to access our data.

In [10]:
    conn = lsst.daf.persistence.Butler(target)
    match = conn.get('visitMatchTable', tract=tract, filter=filter).toDataFrame()['matchId']

In [11]:
    __name__ == '__main__' and display(match.T.iloc[:10, :10], metadata={'match': json.loads(match.head(0).to_json(orient='table'))})

id,42287217204264961,42287217204264962,42287217204264963,42287217204264964,42287217204264965,42287217204264966,42287217204264967,42287217204264968,42287217204264969,42287217204264970
1280,1.099649e+15,1.099649e+15,1.099649e+15,1.099649e+15,1.099649e+15,1.099649e+15,1.099649e+15,1.099649e+15,1.099649e+15,1.099649e+15
1282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1290,1.108162e+15,1.108162e+15,1.108162e+15,1.108162e+15,1.108162e+15,1.108162e+15,1.108162e+15,1.108162e+15,NaN,1.108166e+15
24522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1302,1.118620e+15,1.118620e+15,1.118620e+15,1.118620e+15,1.118620e+15,1.118620e+15,1.118620e+15,1.118620e+15,1.118620e+15,1.118620e+15
24536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
    parquet = conn.get(table, tract=tract, filter=filter)
    raw = pandas.concat([
        parquet.toDataFrame(columns=__annotations__['metric']+__annotations__['flag']), 
        lsst.qa.explorer.functors.CompositeFunctor(composite)(parquet)
    ], axis=1)
    raw[__annotations__['flag']] = raw[__annotations__['flag']].astype(int)
    raw = raw.set_index(list(composite.keys())+__annotations__['flag'], append=True)
    if __name__ == '__main__':
        display(raw.T, metadata={'raw': json.loads(raw.head(0).to_json(orient='table'))})

id,42287217204264961,42287217204264962,42287217204264963,42287217204264964,42287217204264965,42287217204264966,42287217204264967,42287217204264968,42287217204264969,42287217204264970,...,42288351075661679,42288351075661680,42288351075661681,42288351075661682,42288351075661683,42288351075661684,42288351075661685,42288351075661686,42288351075661687,42288351075661688
label,null,null,null,null,null,null,null,null,null,null,...,null,null,null,null,star,null,galaxy,star,star,galaxy
ra,217.132928,217.125231,217.113521,217.112401,217.101904,217.098266,217.095513,217.092807,217.078018,217.060337,...,215.500755,215.501304,215.647176,215.646990,215.549320,215.549725,215.530716,215.531105,215.530274,215.531493
dec,-0.095521,-0.095848,-0.095755,-0.095662,-0.095849,-0.095849,-0.095756,-0.095429,-0.095849,-0.095663,...,1.419002,1.418999,1.422165,1.422025,1.439279,1.439212,1.440442,1.440464,1.440535,1.440431
calib_psf_used,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
calib_psf_candidate,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
calib_photometry_reserved,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
merge_measurement_i2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
merge_measurement_i,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
merge_measurement_r2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
merge_measurement_r,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
    visits = pandas.concat({
        object: conn.get('analysisVisitTable', visit=object, tract=tract, filter=filter).toDataFrame(columns=__annotations__['metric']) 
        for object in match.loc[raw.index.get_level_values('id')].pipe(
            lambda df: df[df.any(axis=1)]
        ).columns})
    if __name__ == '__main__':
        display(visits.T, metadata={'visits': json.loads(visits.head(0).to_json(orient='table'))})


1258                     \
id                             1080845699907585  1080845699907586    
base_Footprint_nPix                   315.000000         41.000000   
Gaussian-PSF_magDiff_mmag            -206.819345       -685.231110   
CircAper12pix-PSF_magDiff_mmag               NaN               NaN   
Kron-PSF_magDiff_mmag                        NaN               NaN   
traceSdss_pixel                         1.659893         77.390163   
traceSdss_fwhm_pixel                    3.908750        182.239906   
psfTraceSdssDiff_percent               19.751169        193.076311   
e1ResidsSdss_milli                    -52.130269        950.151015   
e2ResidsSdss_milli                    -15.569402       -253.137706   

                                                                    \
id                             1080845699907587  1080845699907588    
base_Footprint_nPix                    36.000000         93.000000   
Gaussian-PSF_magDiff_mmag             378.604393         44.862122   
CircAper12pix-PSF_magDiff_mmag               NaN               NaN   
Kron-PSF_magDiff_mmag                        NaN               NaN   
traceSdss_pixel                        83.287455          2.453892   
traceSdss_fwhm_pixel                  196.126968          5.778475   
psfTraceSdssDiff_percent              193.558127         56.907778   
e1ResidsSdss_milli                    959.161386       -146.677318   
e2ResidsSdss_milli                   -268.903002       -460.268647   

                                                                    \
id                             1080845699907589  1080845699907590    
base_Footprint_nPix                    63.000000       1122.000000   
Gaussian-PSF_magDiff_mmag            -890.872261       -689.780782   
CircAper12pix-PSF_magDiff_mmag               NaN               NaN   
Kron-PSF_magDiff_mmag                        NaN               NaN   
traceSdss_pixel                         4.032697          2.459004   
traceSdss_fwhm_pixel                    9.496276          5.790512   
psfTraceSdssDiff_percent               98.732585         57.067333   
e1ResidsSdss_milli                    185.578986       -321.955980   
e2ResidsSdss_milli                   -204.354448        -62.038125   

                                                                    \
id                             1080845699907591  1080845699907592    
base_Footprint_nPix                   444.000000        104.000000   
Gaussian-PSF_magDiff_mmag                    NaN       -255.505501   
CircAper12pix-PSF_magDiff_mmag       -471.464349               NaN   
Kron-PSF_magDiff_mmag                -452.947753       -186.300884   
traceSdss_pixel                         4.633819          1.712854   
traceSdss_fwhm_pixel                   10.911809          4.033463   
psfTraceSdssDiff_percent              108.838608         22.386908   
e1ResidsSdss_milli                   -332.083058         84.293010   
e2ResidsSdss_milli                    730.512345       -294.766688   

                                                                    ...  \
id                             1080845699907593  1080845699907594   ...   
base_Footprint_nPix                   210.000000        683.000000  ...   
Gaussian-PSF_magDiff_mmag            -545.072272       -635.345602  ...   
CircAper12pix-PSF_magDiff_mmag       -942.877067       -847.396385  ...   
Kron-PSF_magDiff_mmag                -713.095223       -736.442595  ...   
traceSdss_pixel                         2.252122          2.265475  ...   
traceSdss_fwhm_pixel                    5.303342          5.334785  ...   
psfTraceSdssDiff_percent               48.888725         49.578694  ...   
e1ResidsSdss_milli                   -359.456520        -26.668015  ...   
e2ResidsSdss_milli                      6.937365       -184.519734  ...   

                                           24538                    \
id                             21078423883482875 21078423883482

In [27]:
    if __name__ == '__main__':
        raw.to_hdf(F'{tract}.h5', 'raw')
        psf.to_hdf(F'{tract}.h5', 'psf')
        visits.to_hdf(F'{tract}.h5', 'visits')

In [28]:
    !ls -l 9615.h5

-rw-r--r-- 1 tonyfast tonyfast 1352339456 Jul 10 16:42 9615.h5


In [29]:
    __name__ == '__main__' and pandas.read_hdf(F'{tract}.h5', 'raw').sample(10).T

id,42288179276971507,42287242974108683,42288342485705006,42287801319841253,42287367528131165,42288071902764065,42287358938227772,42287389002962202,42288329600823144,42287659585906985
label,galaxy,null,galaxy,star,galaxy,galaxy,galaxy,galaxy,null,galaxy
ra,215.696424,216.945619,215.550393,216.274004,216.765803,215.898134,216.901762,216.891254,215.572287,216.562172
dec,-0.032739,1.211920,1.174911,1.421457,0.625257,1.356955,0.267377,1.476411,0.459334,1.391234
calib_psf_used,0,0,0,0,0,0,0,0,0,0
calib_psf_candidate,0,0,0,0,0,0,0,0,0,0
calib_photometry_reserved,0,0,0,0,0,0,0,0,0,0
merge_measurement_i2,0,0,0,0,0,0,0,0,0,0
merge_measurement_i,1,1,0,0,1,0,0,1,1,0
merge_measurement_r2,0,0,0,0,0,0,0,0,0,0
merge_measurement_r,0,0,1,0,0,0,0,0,0,0


In [30]:
    __name__ == '__main__' and pandas.read_hdf(F'{tract}.h5', 'psf').sample(10).T

id,42287522146972200,42287633816098694,42288325305840600,42287376118065909,42287921578929774,42287930168845442,42287371823089134,42287629521152854,42287354643227267,42287221499259110
label,galaxy,galaxy,galaxy,galaxy,galaxy,galaxy,star,star,galaxy,galaxy
ra,216.681651,216.526373,215.553770,216.888358,216.075696,216.170490,216.901810,216.475346,216.789707,217.049273
dec,1.249343,0.175695,0.293260,1.006403,0.676806,1.180709,0.667579,-0.088358,0.046564,0.090706
calib_psf_used,0,0,0,0,0,0,0,0,0,0
calib_psf_candidate,0,0,0,0,0,0,0,0,0,0
calib_photometry_reserved,0,0,0,0,0,0,0,0,0,0
merge_measurement_i2,0,0,0,0,0,0,0,0,0,0
merge_measurement_i,1,1,0,1,1,1,1,1,0,1
merge_measurement_r2,0,0,0,0,0,0,0,0,0,0
merge_measurement_r,0,0,1,0,0,0,0,0,1,0


In [31]:
    __name__ == '__main__' and pandas.read_hdf(F'{tract}.h5', 'visits').sample(10).T

1258              24494  \
id                             1080974548928552  21040409127946454   
base_Footprint_nPix                   396.000000        258.000000   
Gaussian-PSF_magDiff_mmag           -1020.217199       -375.148977   
CircAper12pix-PSF_magDiff_mmag       -949.104195       -347.571780   
Kron-PSF_magDiff_mmag               -1104.412720       -375.762500   
traceSdss_pixel                         3.235249          2.413877   
traceSdss_fwhm_pixel                    7.618430          5.684245   
psfTraceSdssDiff_percent               81.357610         30.789092   
e1ResidsSdss_milli                   -726.575552         74.888813   
e2ResidsSdss_milli                   -104.573308         44.665928   

                                                             1274   \
id                             21040237329254405 1094473631141521    
base_Footprint_nPix                    64.000000         67.000000   
Gaussian-PSF_magDiff_mmag                    NaN       -496.186720   
CircAper12pix-PSF_magDiff_mmag       -689.511798       -635.061159   
Kron-PSF_magDiff_mmag               -1007.864806       -686.591663   
traceSdss_pixel                         8.573894          2.016425   
traceSdss_fwhm_pixel                   20.189976          4.748318   
psfTraceSdssDiff_percent              132.705235         45.453726   
e1ResidsSdss_milli                   -172.892185        340.451771   
e2ResidsSdss_milli                    -68.241024       -237.489586   

                                           1324              1294   \
id                             1137642347433091  1111580485877977    
base_Footprint_nPix                   225.000000        362.000000   
Gaussian-PSF_magDiff_mmag           -1984.286969       -797.463771   
CircAper12pix-PSF_magDiff_mmag      -2268.033051       -530.120743   
Kron-PSF_magDiff_mmag               -2370.423951               NaN   
traceSdss_pixel                        68.037305          3.667412   
traceSdss_fwhm_pixel                  160.215610          8.636096   
psfTraceSdssDiff_percent              192.304235         64.549231   
e1ResidsSdss_milli                    424.685692       -613.787375   
e2ResidsSdss_milli                    195.566438         -8.193556   

                                                             1270   \
id                             1111747989605506  1091338305014866    
base_Footprint_nPix                  2364.000000         61.000000   
Gaussian-PSF_magDiff_mmag             116.735289       -503.857078   
CircAper12pix-PSF_magDiff_mmag       -918.795408       1005.495340   
Kron-PSF_magDiff_mmag                -929.151114       -344.462454   
traceSdss_pixel                         3.748149          1.940919   
traceSdss_fwhm_pixel                    8.826216          4.570515   
psfTraceSdssDiff_percent               72.497065         38.779569   
e1ResidsSdss_milli                    302.110066       -206.820251   
e2ResidsSdss_milli                    227.720801        405.219525   

                                           24522             1262   
id                             21064271966241309 1084294558648913   
base_Footprint_nPix                   132.000000         55.000000  
Gaussian-PSF_magDiff_mmag            -399.930957               NaN  
CircAper12pix-PSF_magDiff_mmag        -49.896816       -369.774635  
Kron-PSF_magDiff_mmag                -307.634170       -526.192763  
traceSdss_pixel                         2.202413          1.852473  
traceSdss_fwhm_pixel                    5.186286          4.362240  
psfTraceSdssDiff_percent               32.100570         51.146512  
e1ResidsSdss_milli                    291.222755       -614.606908  
e2ResidsSdss_milli                   -153.572735        -77.912503

In [32]:
    if __name__ == '__main__':
        !zip tracts.zip 9615.h5

updating: 9615.h5 (deflated 41%)


In [34]:
    !ls -l tracts.zip

-rw-r--r-- 1 tonyfast tonyfast 801038675 Jul 10 16:43 tracts.zip
